# Introduction
The goal of this script is to ease the migration of jobs from one single env/project/account to another env/project/account in dbt Cloud. It excludes Merge Jobs.

Disclaimer :
This is free and unencumbered software released into the public domain.

Anyone is free to copy, modify, publish, use, compile, sell, or
distribute this software, either in source code form or as a compiled
binary, for any purpose, commercial or non-commercial, and by any
means.

In jurisdictions that recognize copyright laws, the author or authors
of this software dedicate any and all copyright interest in the
software to the public domain. We make this dedication for the benefit
of the public at large and to the detriment of our heirs and
successors. We intend this dedication to be an overt act of
relinquishment in perpetuity of all present and future rights to this
software under copyright law.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS BE LIABLE FOR ANY CLAIM, DAMAGES OR
OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,
ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

For more information, please refer to <https://unlicense.org>

### 1. Importing Librairies

In [1]:
import requests
import json
import time
from getpass import getpass

### 2. API Credentials for Base and Target Instance

The following two cells define the API token and account/project IDs for both source and target dbt Cloud environments. These credentials are essential for authentication and specifying which environments to work with. For security best practices, consider using environment variables or secure vaults to handle credentials.




In [2]:
# Define parameters needed for API access on the Source - ACCOUNT YOU ARE MIGRATING FROM
source_api_token = getpass('Source API TOKEN:') # don't edit this, enter practiveley
source_account_id = 70403103916174
source_project_id = 70403103917157
source_environment_id = 70403103918135
source_base_url = 'c1.us1.dbt.com'




Source API TOKEN:··········


In [3]:
# Define parameters needed for API access on the Target - ACCOUNT YOU ARE MIGRATING TO
target_api_token = getpass('Target API TOKEN')
target_account_id = 70403103916174
target_project_id = 70403103917157
target_environment_id =70403103929206
target_base_url = 'c1.us1.dbt.com'


Target API TOKEN··········


### 3. Functions

#### 3.1 Functions to Export Jobs
This function fetches jobs from the source dbt Cloud account and project. It filters jobs based on certain conditions like on_merge and the source environment. The dbt Cloud API endpoint used here is described in their [List Jobs documentation.](https://docs.getdbt.com/dbt-cloud/api-v2#/)

In [4]:
def export_jobs(api_token, account_id, project_id, source_base_url, source_environment_id):
    url = f'https://{source_base_url}/api/v2/accounts/{account_id}/jobs/'
    headers = {
        'Authorization': f'Token {api_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'project_id': int(project_id)
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        jobs_data = response.json()['data']
        #print(jobs_data) Logging initial results

        # Filter out jobs where 'on_merge' is True and environment_id does not match
        filtered_jobs = [job for job in jobs_data if not job['triggers']['on_merge'] and job['environment_id'] == int(source_environment_id)]
        for job in filtered_jobs:
            print(f"On the Environment ID : {job['environment_id']} > Job ID: {job['id']}, Job Name: {job['name']}")  # Print job ID and name and
        #print(filtered_jobs)  # Logging filtered results
        return filtered_jobs
    else:
        raise Exception(f"Failed to fetch jobs: {response.text}")

#### 3.2 Function to Create a Job
This function creates a new job in the target dbt Cloud environment using the jobs data exported from the source. It uses the [Create Job endpoint](https://docs.getdbt.com/dbt-cloud/api-v2#operation/Create%20Job) from the dbt Cloud API.”

In [5]:
def create_job(api_token, account_id, job_data, target_base_url):
    url = f'https://{target_base_url}/api/v2/accounts/{account_id}/jobs/'
    headers = {
        'Authorization': f'Token {api_token}',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, json=job_data)
    if response.status_code == 201:
        return response.json()['data']
    else:
        raise Exception(f"Failed to create job: {response.text}")

#### 3.3 Import Jobs Into Target Env
This function import the created list into the target environment [Create Job endpoint](https://docs.getdbt.com/dbt-cloud/api-v2#operation/Create%20Job) from the dbt Cloud API.

In [6]:
def import_jobs(api_token, target_account_id, target_project_id, target_environment_id, jobs):
    imported_jobs = []
    for job in jobs:
        job_payload = {
            "id" : None,
            "name": job['name'],
            "execute_steps": job['execute_steps'],
            "dbt_version": job['dbt_version'],
            "environment_id":int(target_environment_id),
            "project_id": int(target_project_id),  # Set project ID to the target project
            "account_id": int(target_account_id),
            "state": job['state'],
            "triggers": job['triggers'],
            "schedule": job['schedule'],
            "settings": job['settings']
        }
        #print(job_payload) Use this for the full  input Job Payload
        imported_job = create_job(api_token, target_account_id, job_payload, target_base_url)
        # Logging the data for visibility
        print(f"Job ID: {job['id']}, New Job ID : {imported_job['id']}, Job Name: {job['name']}")
        imported_jobs.append(imported_job)
        time.sleep(1)  # Sleep for 1 second between job creations
    return imported_jobs

### 4. Jobs Migration Execution
Executes the export_jobs function to retrieve and filter jobs from the source dbt Cloud environment.
Executes import_jobs function to import the job in the target env.

In [7]:
# Export Jobs
jobs = export_jobs(source_api_token, source_account_id, source_project_id, source_base_url, source_environment_id)

On the Environment ID : 70403103918135 > Job ID: 70403103917658, Job Name: weekly
On the Environment ID : 70403103918135 > Job ID: 70403103924639, Job Name: وظيفة


In [8]:
# Import Jobs
imported_jobs = import_jobs(target_api_token, target_account_id, target_project_id, target_environment_id, jobs)


Job ID: 70403103917658, New Job ID : 70403103934885, Job Name: weekly
Job ID: 70403103924639, New Job ID : 70403103934886, Job Name: وظيفة


In [9]:
# Summary :
print("Migration completed. Number of jobs migrated:", len(imported_jobs))

Migration completed. Number of jobs migrated: 2
